In [45]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import torch
from torch import nn
from torch.utils import data
from d2l import torch as d2l

关于下标
1. 0~N-1：琪琪的属性嵌入、匹配度、满意度
2. 1~N：whc做的那两个dict文件
3. 真实ID：导出文件格式（题目要求）

In [2]:
# ID映射表
df_per = pd.read_csv('./user_dict',sep='\t')
df_rec = pd.read_csv('./jd_dict',sep='\t')

In [3]:
# 匹配度、满意度，这里的序号是0~N-1
df_ppd = pd.read_csv('./p-u-rating.txt',sep=' ')
df_myd = pd.read_csv('./u-p-rating.txt',sep=' ')
df_ppd.head(), df_myd.head()

(   position  user  ppd
 0         0    94    6
 1         0   165    6
 2         0   168    6
 3         1    94    6
 4         1   165    6,
    user  position  myd
 0   425       295    8
 1   425      1504    8
 2   425      1515    8
 3   425      1542    8
 4   425      1543    8)

In [76]:
def is_fw(pid):
  # return not pid in idx
  val = fw_list.iloc[pid,0]
  return val==1

fw_list = pd.read_csv('./fw.txt',sep=' ',index_col=0)

In [3]:
per_id = {}
for i in df_per.index:
  per_id[df_per.iloc[i,0]] = df_per.iloc[i,1]
rec_id = {}
for i in df_rec.index:
  rec_id[df_rec.iloc[i,0]] = df_rec.iloc[i,1]

In [7]:
for i in range(74):
  print(f'{i} ',end='')

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 

In [24]:
# df_map.to_csv('./per_job_map.csv',header=['job','person','label'],index=None)

In [114]:
# 属性嵌入，这里的序号是0~N-1
a_emb_job = pd.read_csv('./attribute_embedding_position.txt',sep=' ',index_col=0,header=None,na_values=[])
a_emb_per = pd.read_csv('./attribute_embedding_user.txt',sep=' ',index_col=0,header=None,na_values=[])
a_emb_per.index = [per_id(i) for i in range(868)]
print(a_emb_job.shape, a_emb_per.shape)
a_emb_per.head()

(1575, 74) (868, 74)


,1,2,3,4,5,6,7,8,9,10,...,65,66,67,68,69,70,71,72,73,74
0,-0.724494,-0.559137,-0.300785,0.008680,-0.170450,0,0,0.0,1,0,...,-0.160764,0.035278,0.060169,0.088794,-0.306723,0.137945,0.316116,0.4042,0.127718,0.047178
7,-0.724494,-0.721122,-0.890788,4.934121,3.222017,0,0,0.0,1,0,...,-0.160764,0.035278,0.060169,0.088794,-0.306723,0.137945,0.316116,0.4042,0.127718,0.047178
10,1.741550,-0.802114,-0.890788,2.832705,7.543888,0,0,0.0,0,1,...,-0.160764,0.035278,0.060169,0.088794,-0.306723,0.137945,0.316116,0.4042,0.127718,0.047178
12,-0.724494,-0.397152,-0.448285,0.080217,-0.179261,0,0,0.0,1,1,...,-0.160764,0.035278,0.060169,0.088794,-0.306723,0.137945,0.316116,0.4042,0.127718,0.047178
13,-0.724494,3.247509,6.041750,0.025324,-0.260802,0,0,0.0,1,1,...,-0.160764,0.035278,0.060169,0.088794,-0.306723,0.137945,0.316116,0.4042,0.127718,0.047178


先算PU网络，得到匹配度

In [88]:
no_fw = pd.read_csv('./no_fw.csv',index_col=0)
def per_id(x):
  return no_fw.iloc[x,0]

for i in range(10):
  print(per_id(i),end=' ')

0 7 10 12 13 14 15 20 22 23 

In [95]:
# 结构嵌入，这里的序号是0~N-1
d_emb_job = pd.read_csv('./position_pu.csv',index_col=0,header=0,na_values=[])
d_emb_per = pd.read_csv('./user_pu.csv',index_col=0,header=0,na_values=[])
d_emb_per.index = [per_id(i) for i in range(868)]
print(d_emb_job.shape, d_emb_per.shape)
d_emb_per.head()

(1575, 128) (868, 128)


,1,2,3,4,5,6,7,8,9,10,...,119,120,121,122,123,124,125,126,127,128
0,0.106909,-0.046381,0.227989,0.011383,0.049010,0.013847,0.011298,-0.075828,-0.220346,-0.297584,...,0.305304,0.071091,0.004234,0.159778,0.208874,0.261583,0.304411,-0.178296,-0.006043,0.402940
7,0.176486,0.125179,0.170178,0.010691,-0.109280,0.048260,0.103159,-0.049660,-0.293907,-0.443231,...,0.141347,0.170352,0.099519,0.360286,0.144341,0.294506,0.422864,-0.047133,-0.079090,0.512646
10,0.070882,0.197714,0.028229,0.098573,-0.222974,-0.022671,-0.054216,-0.003531,-0.285085,-0.488723,...,-0.069236,0.208382,0.019828,0.413162,0.065986,0.334274,0.341294,0.069452,-0.118390,0.537937
12,0.186596,-0.067015,0.246331,0.018256,0.053443,0.091744,-0.038790,-0.039274,-0.235857,-0.276473,...,0.248856,0.136002,0.001937,0.131124,0.154355,0.266727,0.293040,-0.170457,0.010407,0.351705
13,0.169177,0.083607,0.280480,0.036082,-0.073978,0.088527,0.002985,-0.015185,-0.162121,-0.377745,...,0.212814,0.159757,0.102400,0.217491,0.118156,0.201218,0.407281,-0.038142,-0.057384,0.484031


In [96]:
def get_input(pid, rid):
  # pid = per_id(pid) # 映射到无废物下标
  p_a = a_emb_per.loc[pid].values  # per属性
  p_d = d_emb_per.loc[pid].values  # per结构
  j_a = a_emb_job.loc[rid].values  # job属性
  j_d = d_emb_job.loc[rid].values  # job结构
  all = np.concatenate([p_a,p_d,j_a,j_d],axis=0)
  return all

get_input(1,1).shape

(404,)

In [112]:
# x=get_input(4335,0)
# x.shape
per_id(4335)

IndexError: single positional indexer is out-of-bounds

In [109]:
features = []
labels = []
get_input(1,2).shape

(404,)

In [100]:
df_ppd.head()

,position,user,ppd
0,0,94,6
1,0,165,6
2,0,168,6
3,1,94,6
4,1,165,6


In [110]:
for i in tqdm(df_ppd.index):
  row = df_ppd.loc[i,]
  print(row['user'], row['position'], row['ppd'])
  feature = get_input(row['user'],row['position'])
  label = row['ppd']
  features.append(feature)
  labels.append(label)
  

  0%|          | 90/526221 [00:00<05:34, 1574.78it/s]

94 0 6
165 0 6
168 0 6
94 1 6
165 1 6
168 1 6
94 2 6
165 2 6
168 2 6
94 9 6
165 9 6
168 9 6
94 26 6
165 26 6
168 26 6
94 62 6
165 62 6
168 62 6
94 69 6
165 69 6
168 69 6
94 104 6
165 104 6
168 104 6
94 107 6
165 107 6
168 107 6
94 108 6
165 108 6
168 108 6
94 110 6
165 110 6
168 110 6
94 140 6
165 140 6
168 140 6
94 201 6
165 201 6
168 201 6
94 219 6
165 219 6
168 219 6
94 228 6
165 228 6
168 228 6
94 346 6
165 346 6
168 346 6
94 1507 6
165 1507 6
168 1507 6
94 1508 6
165 1508 6
168 1508 6
94 1529 6
165 1529 6
168 1529 6
94 1574 6
165 1574 6
168 1574 6
49 0 8
49 1 8
49 2 8
49 9 8
49 26 8
49 62 8
49 69 8
49 104 8
49 107 8
49 108 8
49 110 8
49 140 8
49 201 8
49 219 8
49 228 8
49 346 8
49 1507 8
49 1508 8
49 1529 8
49 1574 8
20 0 5
24 0 5
25 0 5
26 0 5
28 0 5
44 0 5
93 0 5
102 0 5
339 0 5
552 0 5
4335 0 5


IndexError: single positional indexer is out-of-bounds

[6, 6, 6, 6, 6]

In [12]:
features = np.array(features)
labels = np.array(labels)
features = torch.Tensor(features)
labels = torch.Tensor(labels)
features.shape

torch.Size([35291, 283])

In [13]:
class DS:
  def __init__(self,data_arrays):
    self.data = data_arrays

  def __getitem__(self,index):
    return self.data[0][index],self.data[1][index]
  
  def __len__(self):
    return self.data[0].size()[0]

def load_array(data_arrays, batch_size, is_train=True):
  # Array to Iter
  dataset = data.TensorDataset(*data_arrays)
  # dataset = DS(data_arrays)
  return data.DataLoader(dataset, batch_size, shuffle=is_train)
    

In [110]:
batch_size = 50
train_size = 30000
test_size = 1000

train_iter = load_array((features[:train_size],labels[:train_size]), batch_size)
test_iter = load_array((features[train_size:train_size+test_size],labels[train_size:train_size+test_size]), batch_size, False)

In [70]:
features.shape

torch.Size([35291, 283])

In [71]:
net = nn.Sequential(nn.Linear(283,200),
                    nn.ReLU(),
                    nn.Linear(200,100),
                    nn.ReLU(),
                    nn.Linear(100,2)
                    )

In [72]:
def init_weights(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, std=0.01)

net.apply(init_weights)

Sequential(
  (0): Linear(in_features=283, out_features=200, bias=True)
  (1): ReLU()
  (2): Linear(in_features=200, out_features=100, bias=True)
  (3): ReLU()
  (4): Linear(in_features=100, out_features=2, bias=True)
)

In [111]:
loss = nn.MSELoss()
# loss = nn.CrossEntropyLoss(reduction='none')
trainer = torch.optim.SGD(net.parameters(), lr=0.01)

In [ ]:
num_epochs = 100
d2l.train_ch3(net, train_iter, test_iter, loss, num_epochs, trainer)

In [128]:
num_epochs = 10
looper = tqdm(range(num_epochs))
for epoch in looper:
    for X, y in train_iter:
        # print(X.shape, y.shape)
        l = loss(net(X) ,y)
        trainer.zero_grad()
        l.backward()
        trainer.step()
    l = loss(net(features[train_size:train_size+test_size]), labels[train_size:train_size+test_size])
    # print(f'epoch {epoch + 1}, loss {l:f}')
    looper.set_description(f'epoch {epoch + 1}, loss {l:f}')

epoch 10, loss 0.013473: 100%|██████████| 10/10 [00:36<00:00,  3.69s/it]


In [165]:
print(net(torch.Tensor(get_input(51121,2044))))
print(net(torch.Tensor(get_input(54525,1075))))
# net.state_dict()

tensor([0.9993, 0.6959], grad_fn=<AddBackward0>)
tensor([1.0001, 0.6275], grad_fn=<AddBackward0>)


In [122]:
torch.save(net,'MLP.pt')